<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Dodanie_top_typ_filmu/colabs/Dane_z_dodatkowymi_kolumnami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1297, done.
remote: Counting objects: 100% (516/516), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 1297 (delta 399), reused 271 (delta 246), pack-reused 781
Receiving objects: 100% (1297/1297), 5.94 MiB | 18.17 MiB/s, done.
Resolving deltas: 100% (747/747), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [4]:
# !git checkout <nazwa_twojego_brancha>

In [5]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 200 kB 47.4 MB/s 
     |████████████████████████████████| 199 kB 13.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=92193c7c0f7a8b96eae8d0df1fa32db064ad5269ee4c9018d59ac9f1a0fe2f9e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-29 19:09:59--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [6]:
import pandas as pd
from lib.pyspark_init import create_spark_context
from lib.adding_columns import add_all_columns
from lib.const import DATA_WITH_AWARDS, TRAIN_DATA, VALIDATE_DATA, TEST_DATA

In [7]:
spark = create_spark_context()
spark

In [8]:
!chmod +x download_data.sh
!./download_data.sh

--2022-12-29 19:10:29--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.40, 13.224.2.93, 13.224.2.111, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238426506 (227M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 227.38M  58.2MB/s    in 3.9s    

2022-12-29 19:10:33 (57.6 MB/s) - ‘name.basics.tsv.gz’ saved [238426506/238426506]

--2022-12-29 19:10:33--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.224.2.40, 13.224.2.93, 13.224.2.111, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.224.2.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291823241 (278M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 278.30M  58.8MB/s   

In [9]:
%%time
data = spark.read.parquet(DATA_WITH_AWARDS)
data.show()

CPU times: user 72.1 ms, sys: 7.49 ms, total: 79.6 ms
Wall time: 10.3 s


In [10]:
%%time
data = add_all_columns(spark, data)
data.show()

CPU times: user 154 ms, sys: 28.6 ms, total: 183 ms
Wall time: 6.92 s


## Dodanie top type dla aktora

In [11]:
from pyspark.sql.functions import explode, split
from pyspark.sql import Window
from pyspark.sql import Window
from pyspark.sql.functions import count, row_number, desc, first

In [12]:
df2 = data.select('nconst', explode(data.titleType).alias('titleType'))
df3 = df2.select('nconst', explode(split(df2.titleType, ',')).alias('titleType'))

w = Window.partitionBy('nconst', 'titleType')
aggregated_table = df3.withColumn("count",count("*").over(w)).withColumn("rn",
                                                                 row_number().over(
                                                                     w.orderBy(desc("count")))).filter(
                                                                         "rn = 1").groupBy('nconst').agg(
                                                                             first('titleType').alias('top_type'))
#aggregated_table.show()

In [13]:
data = data.join(aggregated_table, data.nconst==aggregated_table.nconst, 'left').drop(aggregated_table.nconst)
#data.show()

## Dodanie top gatunku

In [15]:
df4 = data.select('nconst', explode(data.genres).alias('genres'))
df5 = df4.select('nconst', explode(split(df4.genres, ',')).alias('genres'))

w_1 = Window.partitionBy('nconst', 'genres')
aggregated_table_2 = df5.withColumn("count",count("*").over(w_1)).withColumn("rn",
                                                                 row_number().over(
                                                                     w_1.orderBy(desc("count")))).filter(
                                                                         "rn = 1").groupBy('nconst').agg(
                                                                             first('genres').alias('top_genres'))
#aggregated_table_2.show()

In [17]:
data = data.join(aggregated_table_2, data.nconst==aggregated_table_2.nconst, 'left').drop(aggregated_table_2.nconst)
data.show()

+--------------------+--------------------+--------------------+--------------------+---------+-------+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-------------+------------+------------+-------------+--------+---------------------+---------+---------------------+---------+-------------------+-------+--------+--------------------+---------+---------+----------+
|              tconst|           titleType|       originalTitle|             isAdult|startYear|endYear|              genres|category|          characters|         primaryName|      knownForTitles|         year_oscars|     category_oscars|         film_oscars|       winner_oscars|         year_globes|     category_globes|         film_globes|          win_globes|year_emmy|category_emmy|nominee_emmy|company_

## Podział zbioru

In [ ]:
%%time
train_data, validate_data, test_data = data.randomSplit([0.7, 0.1, 0.2], seed=1)

In [ ]:
%%time
data.write.parquet(TRAIN_DATA)

In [ ]:
%%time
data.write.parquet(VALIDATE_DATA)

In [ ]:
%%time
data.write.parquet(TEST_DATA)